In [31]:
import numpy as np

In [32]:
import pandas as pd

In [33]:
d = pd.read_csv('./GAM.csv', sep = ';', decimal = ',')
d

,y,x1,x2,x3,x4
0,12345.0,2769,29.67,94.2,9.6
1,12091.9,3065,28.94,101.1,9.5
2,12339.7,3899,28.43,111.6,9.5
3,12450.6,3790,27.50,119.7,9.6
4,12832.1,3963,28.07,112.0,9.6
5,12892.1,4224,28.08,112.0,9.4
6,13224.5,4645,27.68,115.3,9.0
7,13449.9,4914,28.86,109.6,8.2
8,13476.8,4830,31.88,112.6,7.2
9,13588.9,4815,29.90,108.7,7.2


In [34]:
d.dtypes

y     float64
x1      int64
x2    float64
x3    float64
x4    float64
dtype: object

In [35]:
import statsmodels.formula.api as sm

In [36]:
m = sm.ols(formula="y ~ x1 + x2 + x3 +x4", data=d)
results = m.fit()
s = results.summary()
s

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.806
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     17.70
Date:                Wed, 13 Oct 2021   Prob (F-statistic):           6.83e-06
Time:                        13:37:25   Log-Likelihood:                -168.61
No. Observations:                  22   AIC:                             347.2
Df Residuals:                      17   BIC:                             352.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    1.04e+04   6169.698      1.686      0.110   -2616.991    2.34e+04
x1             0.4838      0.196      2.463      0.025       0.069       0.898
x2           130.3151    132.126      0.986      0.338    -148.446     409.076
x3            -6.4678     25.698     -0.252      0.804     -60.686      47.750
x4          -276.3727     78.946     -3.501      0.003    -442.935    -109.810
==============================================================================
Omnibus:                        5.043   Durbin-Watson:                   1.083
Prob(Omnibus):                  0.080   Jarque-Bera (JB):                2.978
Skew:                           0.808   Prob(JB):                        0.226
Kurtosis:                       3.797   Cond. No.                     2.32e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [37]:
import statsmodels.stats as smd

import statsmodels.stats.api as sms
#чтобы делать тесты

from statsmodels.compat import lzip
#чтобы делать подписи

In [38]:
residuals = results.resid

#DWtest
dw = smd.stattools.durbin_watson(residuals)
dw

#BGtest
smd.diagnostic.acorr_breusch_godfrey(results)
#или
name=['Chisq statistic','p-value', 'F statistic','p-value(F)']
bg=sms.acorr_breusch_godfrey(results,nlags=1)
lzip(name,bg)

[('Chisq statistic', 3.5386954765277334),
 ('p-value', 0.05995259588586508),
 ('F statistic', 3.066908275764395),
 ('p-value(F)', 0.09904494651386497)]

In [39]:
#GQtest
name=['F statistic','p-value']
gq=sms.het_goldfeldquandt(residuals,results.model.exog)
lzip(name,gq)
#или GQtest
gq=sms.het_goldfeldquandt(residuals,results.model.exog)
gq


(12.06478948111151, 0.003985192273289891, 'increasing')

In [40]:
#BPtest
name=['BP','p-value']
bp=sms.het_breuschpagan(residuals,results.model.exog)
lzip(name,bp)


[('BP', 4.349947045310061), ('p-value', 0.3607124304434168)]

In [41]:
#White_test
name=['WT','p-value']
wt=sms.het_white(residuals,results.model.exog)
lzip(name,wt)

[('WT', 15.049318977211422), ('p-value', 0.37479162866559157)]

In [42]:
#HC
smd.sandwich_covariance.cov_hc3(results)
#или
results.cov_HC3


/opt/homebrew/lib/python3.9/site-packages/statsmodels-0.14.0.dev0-py3.9-macosx-11-arm64.egg/statsmodels/stats/sandwich_covariance.py:169: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  scale[:,None]*results.model.pinv_wexog.T)


array([[ 4.42043696e+07,  4.29619101e+02, -9.89688378e+05,
        -1.24626777e+05, -4.54252382e+05],
       [ 4.29619101e+02,  2.76061969e-02, -1.26356042e+01,
        -1.65214932e+00,  1.62610075e-01],
       [-9.89688378e+05, -1.26356042e+01,  2.28869356e+04,
         2.73715523e+03,  9.81389140e+03],
       [-1.24626777e+05, -1.65214932e+00,  2.73715523e+03,
         4.03241541e+02,  1.02302105e+03],
       [-4.54252382e+05,  1.62610075e-01,  9.81389140e+03,
         1.02302105e+03,  6.95617787e+03]])

In [43]:
#HAC
smd.sandwich_covariance.cov_hac(results)


array([[ 4.56271457e+07,  1.19502473e+02, -1.00255190e+06,
        -1.22267811e+05, -4.86988784e+05],
       [ 1.19502473e+02,  2.01909042e-02, -5.63541897e+00,
        -3.26284914e-01, -3.68608033e-01],
       [-1.00255190e+06, -5.63541897e+00,  2.26757158e+04,
         2.62638290e+03,  1.06648611e+04],
       [-1.22267811e+05, -3.26284914e-01,  2.62638290e+03,
         3.49538163e+02,  1.23966636e+03],
       [-4.86988784e+05, -3.68608033e-01,  1.06648611e+04,
         1.23966636e+03,  5.83059052e+03]])